In [1]:
from Flamingo.utils.pretty import pretty_print, print_local_vars, vis_model
from Flamingo.utils.test_model import MyClassifier
from loguru import logger
from peft import LoraConfig, PeftModel, LoraModel, LoraConfig, get_peft_model
import torch 
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from rich import print 
config = LoraConfig(r=32, lora_alpha=64, target_modules=["q", "v"], lora_dropout=0.05, bias="none")

[2023-12-20 18:44:50,946] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
model_id="google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

In [3]:
peft_model = get_peft_model(model=model, peft_config=config)
peft_model.print_trainable_parameters()
print(peft_model)

trainable params: 1,376,256 || all params: 78,337,408 || trainable%: 1.7568311680672406


PeftModel(
  (base_model): LoraModel(
    (model): T5ForConditionalGeneration(
      (shared): Embedding(32128, 512)
      (encoder): T5Stack(
        (embed_tokens): Embedding(32128, 512)
        (block): ModuleList(
          (0): T5Block(
            (layer): ModuleList(
              (0): T5LayerSelfAttention(
                (SelfAttention): T5Attention(
                  (q): lora.Linear(
                    (base_layer): Linear(in_features=512, out_features=384, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=512, out_features=32, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=32, out_features=384, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
                    (lora_embedding_B): ParameterDict()
                  )
                  (k): Linear(in_features=512, out_features=384, bias=False)
                  (v): lora.Linear(
                    (base_layer): Linear(in_features=512, out_features=384, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=512, out_features=32, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=32, out_features=384, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
                    (lora_embedding_B): ParameterDict()
                  )
                  (o): Linear(in_features=384, out_features=512, bias=False)
                  (relative_attention_bias): Embedding(32, 6)
                )
                (layer_norm): T5LayerNorm()
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (1): T5LayerFF(
                (DenseReluDense): T5DenseGatedActDense(
                  (wi_0): Linear(in_features=512, out_features=1024, bias=False)
                  (wi_1): Linear(in_features=512, out_features=1024, bias=False)
                  (wo): Linear(in_features=1024, out_features=512, bias=False)
                  (dropout): Dropout(p=0.1, inplace=False)
                  (act): NewGELUActivation()
                )
                (layer_norm): T5LayerNorm()
                (dropout): Dropout(p=0.1, inplace=False)
              )
            )
          )
          (1-7): 7 x T5Block(
            (layer): ModuleList(
              (0): T5LayerSelfAttention(
                (SelfAttention): T5Attention(
                  (q): lora.Linear(
                    (base_layer): Linear(in_features=512, out_features=384, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=512, out_features=32, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=32, out_features=384, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
                    (lora_embedding_B): ParameterDict()
                  )
                  (k): Linear(in_features=512, out_features=384, bias=False)
                  (v): lora.Linear(
                    (base_layer): Linear(in_features=512, out_features=384, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=512, out_features=32, bias=False)
            

In [4]:
peft_model.peft_config['default'].bias
peft_model.named_parameters()

<generator object Module.named_parameters at 0x7fcbaa340040>

In [3]:
from deepspeed import zero 
from deepspeed.runtime.zero.partition_parameters import ZeroParamStatus
def _z3_params_to_fetch(param_list):
    return [
        p for p in param_list
        if hasattr(p, 'ds_id') and p.ds_status == ZeroParamStatus.NOT_AVAILABLE
    ]
def maybe_zero_3(param):
    """ 
        zero stage == 3 will partition the model weight to different rank
    """
    if hasattr(param, "ds_id"):
        assert param.ds_status == ZeroParamStatus.NOT_AVAILABLE
        with zero.GatheredParameters([param]):
            param = param.data.detach().cpu().clone()
    else:
        param = param.detach().cpu().clone()
    return param

# Borrowed from peft.utils.get_peft_model_state_dict
def get_peft_state_maybe_zero_3(named_params, bias):
    """ 
        reference: DeepSpeed FastChat:
            https://github.com/lm-sys/FastChat/blob/4960ca702c66b9adaa65945746dba34f8d2c8ddc/fastchat/train/train_lora.py#L66
        only save huggingface Peft model's LoRA weight:
            1) lora_A
            2) lora_B
            3) lora_dropout
            4) lora_embedding_A
            5) lora_embedding_B
        usage:

    """
    if bias == "none":
        to_return = {k: t for k, t in named_params if "lora_" in k}
    elif bias == "all":
        to_return = {k: t for k, t in named_params if "lora_" in k or "bias" in k}
    elif bias == "lora_only":
        to_return = {}
        maybe_lora_bias = {}
        lora_bias_names = set()
        for k, t in named_params:
            if "lora_" in k:
                to_return[k] = t
                bias_name = k.split("lora_")[0] + "bias"
                lora_bias_names.add(bias_name)
            elif "bias" in k:
                maybe_lora_bias[k] = t
        for k, t in maybe_lora_bias:
            if bias_name in lora_bias_names:
                to_return[bias_name] = t
    else:
        raise NotImplementedError
    to_return = {k: maybe_zero_3(v) for k, v in to_return.items()}
    return to_return

In [5]:

# print(model)

In [2]:
# a = torch.Tensor([1,2,3,4]) / 2
# a = a.cuda()

In [6]:
# a.to(torch.bfloat16)

tensor([0.5000, 1.0000, 1.5000, 2.0000], device='cuda:0', dtype=torch.bfloat16)

In [3]:
# model = MyClassifier(3, 100, ch_multi=128)

In [6]:
# x = vis_model(model)

In [7]:
# logger.info(model)

In [25]:

# print(x)

stage1 (Sequential)
├── 0 (Sequential)
│   ├── 0 (Conv2d) weight:[128, 3, 2, 2] bias:[128]
│   └── 1 (GroupNorm) weight:[128] bias:[128]
└── 1-2(ConvBlock) layer_scale:[]
    └── residual (Sequential)
        ├── 0-0,3-3(GroupNorm) weight:[128] bias:[128]
        └── 2-2,5-5(Conv2d) weight:[128, 128, 3, 3] bias:[128]

a decorator to print local varibles in function scope

In [4]:
a = dict(
    a=0,
    b="hello",
    c=[1,2,3]
)
b = "hellow world"

In [5]:
@print_local_vars
def function_scope(a, b):
    return a['b'] + "   " + b
function_scope(a, b)

Local ! 🧛
{
    'args': ({'a': 0, 'b': 'hello', 'c': [1, 2, 3]}, 'hellow world'),
    'kwargs': {},
    'result': 'hello   hellow world',
    'func': <function function_scope at 0x7fdde7a2de50>
}

'hello   hellow world'

In [1]:
from loguru import logger

logger.debug("That's it, beautiful and simple logging!")

2023-12-16 22:48:33.612 | DEBUG    | __main__:<module>:3 - That's it, beautiful and simple logging!


In [2]:
logger.add("file_{time}.log")

1

In [8]:
work_dir = "logg"
logger.add(work_dir + "/{time}.log", enqueue=True)
# logger.add(sink="logg/{}file_{}.log".format("test"), enqueue=True)

4

In [3]:
logger.debug("That's it, beautiful and simple logging!")

2023-12-16 22:52:37.650 | DEBUG    | __main__:<module>:1 - That's it, beautiful and simple logging!


In [4]:
logger.info("Hello world")

2023-12-16 22:57:55.204 | INFO     | __main__:<module>:1 - Hello world
